# Programming for A&A

### Lecture moment 6: Scipy and selecting data from arrays

## Scipy
Scipy is a collection of packages based on numpy, which enable numerical calculations, statistical data analysis and signal processing amongst other things. 

Here we will focus on using Scipy's `optimize` to fit models to data. Along the way we will also learn more about using conditional statements to select data from arrays.

In [ ]:
# First we will import the packages we need:
import numpy as np
import matplotlib.pyplot as plt
# We need to load each scipy package individually (a package is a collection of functions):
import scipy.optimize as spopt

## The bolometric luminosity functions of quasars
We're going to take a look at the data in the text file `quasar_bol_lumfunc.dat`, which gives the observed _bolometric luminosity functions_ of quasars (rapidly-accreting supermassive black holes) for different redshifts. 
* _Bolometric_ means the total luminosity (in solar luminosities)
* A _luminosity function_ is a histogram of the number of objects per unit volume and luminosity. 

First let's take a look at the file:

In [ ]:
with open('quasar_bol_lumfunc.dat', 'r') as f: 
    print(f.read())

## Reading in the data
We'll now read in the data we need, which is just the first four columns. Since column names are not given, the data are read into a 2-D array. For convenience we will then assign each column to a separate 1-D array:

In [ ]:
qsr_lumfun = np.genfromtxt('quasar_bol_lumfunc.dat',comments=';',usecols=[0,1,2,3])
print(qsr_lumfun)
z, loglum_sol, lf, lf_err = qsr_lumfun[:,0], qsr_lumfun[:,1], qsr_lumfun[:,2], qsr_lumfun[:,3]

## Selecting on redshift and plotting
* We can use conditional statements to select data with certain properties, e.g. the redshift `z`. 
* Plotting the data for given redshifts, we see that some data have much larger errors than others. We can remove these data with a conditional statement.

In [ ]:
plt.figure(figsize=[10,7])
err_lim = 0.2  # Choose maximum error bar size allowed
for zval in [0.5, 1.0, 2.0]:
    # Note that if we want to combine different conditions we need to put them in parentheses:
    plt.errorbar(loglum_sol[(z==zval) & (lf_err < err_lim)], lf[(z==zval) & (lf_err < err_lim)], 
                 yerr=lf_err[(z==zval) & (lf_err < err_lim)], label='z = '+str(zval), linestyle='')
plt.xlabel(r'$\log(L_{\rm bol}/L_{\odot})$',fontsize=14)
plt.ylabel(r'${\rm d}\log \phi/{\rm d}\log(L_{\rm bol})$ (Mpc$^{-3}$ $\log(L_{\rm bol})^{-1})$',fontsize=14)
plt.legend(fontsize=16)
plt.show()

## Defining a model to fit the data
* The data look like they might be fitted by a straight lines with a change in gradient at a 'break'. Let's define a function for this model, so we can fit it...
* Note that the ordering of function parameters is set by the `curve_fit` function we will use to fit it ([check the documentation for curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html)): 
    * The data x-values go first, then the function parameters. The function only outputs the model y-values

In [ ]:
def bkn_lin(xvals,grad1,grad2,bk,y_bk):
    '''Linear model with a break:
        Inputs: xvals: x-values, grad1/grad2 gradient below/above break, bk: x-value for break
        y_bk: y-value at break (sets the model normalisation)
        Returns: mod_yvals: model y-values for given x-values and parameters'''
    
    mod_yvals = np.zeros(len(xvals)) # We will not define and fill array in one step, so need to define it first

    # Choose linear model based on whether x-value below/above break 
    # The condition sets the gradient we use. The intercept at the break is the same in both cases.
    mod_yvals[xvals < bk] = grad1*(xvals[xvals < bk]-bk) + y_bk
    mod_yvals[xvals >= bk] = grad2*(xvals[xvals >= bk]-bk) + y_bk
    return mod_yvals

## Check the model
* Before fitting the model you should always plot it to check that it works and seems suitable to describe the data!
* We can also use a visual comparison to choose some plausible starting parameters for the fit.

In [ ]:
zval, err_lim = 2.0, 0.2  # Choose the redshift and error values to select the data to compare with the model
x_model = np.linspace(10,16,100) # The model is continuous so we should not just use the data values to plot
                                 # or it will look 'wrong'. So we define a grid of x-values to evaluate it on

plt.figure(figsize=[10,7])
plt.errorbar(loglum_sol[(z==zval) & (lf_err < err_lim)], lf[(z==zval) & (lf_err < err_lim)], 
                 yerr=lf_err[(z==zval) & (lf_err < err_lim)], label='z = '+str(zval), linestyle='')
plt.plot(x_model,bkn_lin(x_model,-1,-2,13,-5),label='model')    
plt.xlabel(r'$\log(L_{\rm bol}/L_{\odot})$',fontsize=14)
plt.ylabel(r'${\rm d}\log \phi/{\rm d}\log(L_{\rm bol})$ Mpc$^{-3}$ $\log(L_{\rm bol})^{-1}$',fontsize=14)
plt.legend(fontsize=16)
plt.show()

## Fitting the model
* We use `scipy.optimize.curve_fit` to fit the data including its errors (these 'weight' the fit: see the Statistical Methods course for more details!)
* The function needs a 'guess' for its starting parameters, which we give as the list p0. Your guess needs to be good enough that the fitting function can find the best fit. It does this by trying to minimise the _chi-squared_ : the total squared difference of the y-values of the data and model (weighted by errors).
* If you choose starting parameters that are far from the best-fitting ones, the function may not find the best-fit (it may get stuck in a local minimum of chi-squared). Smooth model functions are more forgiving than those with sharp edges/features!

In [ ]:
# For convenience we will pre-select our data into 1-D arrays:
zval, err_lim = 1.0, 0.2
xdata, ydata, yerr = loglum_sol[(z==zval) & (lf_err < err_lim)], lf[(z==zval) & (lf_err < err_lim)],\
                            lf_err[(z==zval) & (lf_err < err_lim)]
# Our earlier plotted model was not too bad, so we will use it for the starting parameters
p0 = [-1,-2,13,-5]
params, params_covariance = spopt.curve_fit(bkn_lin, xdata, ydata, p0, sigma=yerr)
# The function outputs two arrays with the best-fitting parameters and the covariance matrix, which give an estimate
# of the errors on the parameters (and the correlated errors between them). The square-root of the diagonal values 
# of the covariance matrix array corresponds to 'traditional' 1-sigma error bars. 
print("Best-fitting parameters, errors = ",params, np.sqrt(np.diag(params_covariance)))

## Plot the data and best-fitting model
Finally plot the best-fitting model and the data to check that the fit looks good!

In [ ]:
plt.figure(figsize=[10,7])
plt.errorbar(xdata,ydata,yerr=yerr,label='z = '+str(zval), linestyle='')
# Note a handy trick: in a function call you can 'unpack' a list into separate parameters, to fit the
# required function definition, using the * prefix:
plt.plot(x_model,bkn_lin(x_model,*params),label='model')    
plt.xlabel(r'$\log(L_{\rm bol}/L_{\odot})$',fontsize=14)
plt.ylabel(r'${\rm d}\log \phi/{\rm d}\log(L_{\rm bol})$ Mpc$^{-3}$ $\log(L_{\rm bol})^{-1}$',fontsize=14)
plt.legend(fontsize=16)
plt.show()